# 🧠 Notebook: 02 ETL Gold Layer

This notebook implements the **Gold Layer (AI-Powered Analytics)** of the solution accelerator, transforming transcribed call data into enriched insights using **Databricks AI Functions** and **provisionless batch inference**.

It introduces structured AI outputs like sentiment, summaries, classifications, named entities, and even generates professional follow-up emails — ready for downstream workflows or customer engagement.

---

## 🧱 Purpose

To apply **advanced AI inference** on transcribed customer calls and output enriched, actionable insights in a format that can be:
- Embedded in dashboards
- Trigger customer communications
- Drive operational decisions

In [0]:
%run "./resources/init" 

In [0]:
prompt = """Using the following call transcript, generate a professional yet friendly email from the agent to the customer. The email should summarize the key points of the conversation, including the reason for the call, the resolution provided, and any necessary next steps for the customer. The tone should be courteous, clear, and supportive.

Email Structure:
- Subject Line: A clear and concise subject summarizing the purpose of the email (e.g., “Follow-up on Your Prescription Claim - VitalGuard Health Insurance”).
- Greeting: A friendly yet professional greeting addressing the customer by name.
- Call Summary: A recap of the discussion, including the inquiry and the response of the agent.
- Next Steps: A clear outline of any actions the customer needs to take (e.g., contacting their doctor, submitting forms, waiting for updates).
- Contact Information: An invitation for the customer to reach out if they have further questions.
- Closing: A polite and professional closing with the name of the agent and company details.

Call Transcript:
\n
"""

response_format = '''{
    "type": "json_schema",
    "json_schema": {
        "name": "vitalguard_call_followup_email",
        "schema": {
            "type": "object",
            "properties": {
                "subject": {
                    "type": "string",
                    "description": "The subject line of the email summarizing the purpose of the follow-up."
                },
                "greeting": {
                    "type": "string",
                    "description": "A friendly yet professional greeting addressing the customer by name."
                },
                "call_summary": {
                    "type": "string",
                    "description": "A summary of the inquiry of the customer and the response given by the agent."
                },
                "next_steps": {
                    "type": "string",
                    "description": "Clear and concise next steps that the customer needs to take, if applicable."
                },
                "contact_information": {
                    "type": "string",
                    "description": "Details on how the customer can reach out for further assistance."
                },
                "closing": {
                    "type": "string",
                    "description": "A polite closing statement including the name of the agent and company details."
                }
            },
            "required": [
                "subject",
                "greeting",
                "call_summary",
                "next_steps",
                "contact_information",
                "closing"
            ]
        },
        "strict": true
    }
}'''

def create_sql_array(array):
    return ", ".join([f"'{item}'" for item in array])

reasons_for_call_list = [row['reason_for_call'] for row in spark.table(f"{CATALOG}.{SCHEMA}.call_centre_reasons").select("reason_for_call").distinct().collect()]
reasons_for_call_categories = create_sql_array(reasons_for_call_list) 

ner_list = ["firstName_lastName", "dateOfBirth_yyyy-mm-dd", "policy_number"]
ner = create_sql_array(ner_list)

In [0]:
df = spark.table(f"{CATALOG}.{SCHEMA}.transcriptions_silver")
df.createOrReplaceTempView("transcriptions_temp")

query = f"""
    SELECT *,
          ai_analyze_sentiment(transcription) AS sentiment,
          ai_summarize(transcription) AS summary,
          ai_classify(transcription, ARRAY({reasons_for_call_categories})) AS classification,
          ai_extract(transcription, ARRAY({ner})) AS ner,
          ai_query('{ENDPOINT_NAME}', CONCAT('{prompt}', transcription), responseFormat => '{response_format}') AS email_response
    FROM transcriptions_temp
"""

transcriptions_with_ai = spark.sql(query)

transcriptions_with_ai.createOrReplaceTempView("transcriptions_temp_1")

query_1 = f"""
    SELECT *
          , ai_mask(summary, ARRAY('person', 'address')) AS summary_masked
    FROM transcriptions_temp_1
"""

transcriptions_final = spark.sql(query_1)

display(transcriptions_final)

In [0]:
from pyspark.sql.functions import col

final_df = transcriptions_final.withColumn("customer_name", col("ner.firstName_lastName")) \
       .withColumn("birth_date", col("ner.dateOfBirth_yyyy-mm-dd")) \
       .withColumn("policy_number", col("ner.policy_number")) \
       .drop("path", "modificationTime", "file_path", "file_name", "transcription", "ner")

display(final_df)

final_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(f"{CATALOG}.{SCHEMA}.analysis_gold")

## ✅ Output
**Table: analysis_gold**

Includes:
- AI sentiment classification
- Summaries (original and masked)
- Call reason classification
- Extracted customer info
- Structured JSON for follow-up emails

## ⏭ Next Step

Consume this enriched dataset in:
- **Dashboards** (e.g., sentiment trends, fraud alerts, agent metrics)
- **Case management systems**
- **Automated email or notification APIs**